# RNA-Seq classification

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from scipy import stats
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, f1_score
np.random.seed(12345678)

In [ ]:
all_rna = pd.read_csv('../RNA-Seq/RNA-ExpAll-LC.csv.gz', compression='gzip')

In [ ]:
classes = np.array(['healthy', 'adeno', 'squa'])

ohe = preprocessing.OneHotEncoder(sparse=False)
ohe.fit(classes.reshape(-1,1))

In [ ]:
def train(x_train, x_test, y_train_ohe, y_test_ohe, accs, f1s, name, early=False, verbose=False):
    
    tuned_parameters = [{'max_depth': [2, 4, 6, 8],
                            'n_estimators': [20, 30, 50, 100, 200],
                         'alpha': [0,0.1,0.2,0.3]}]
    
    
    clf = GridSearchCV(
                    xgb.XGBClassifier(n_jobs=1,use_label_encoder=False,verbosity = 0, random_state=42), tuned_parameters, 
                    scoring='accuracy'
                )
    if early:
        classes_ = [0,1,2]
        x_train_new, y_train_ohe_new, x_val, y_val = get_val_set(x_train, y_train_ohe, classes_, percentage = 0.1)
    else:
        x_train_new = x_train
        y_train_ohe_new = y_train_ohe
    
    scaler = StandardScaler()
    x_train_new = scaler.fit_transform(x_train_new)
    if early:
        eval_set=[(scaler.transform(x_val), y_val.argmax(axis=1))]
        clf.fit(x_train_new, y_train_ohe_new.argmax(axis=1), early_stopping_rounds=10, eval_set=eval_set, verbose=False)
    else:
        clf.fit(x_train_new, y_train_ohe_new.argmax(axis=1))
    print(clf.best_params_)
    best_params = clf.best_params_
    train_preds = clf.predict(scaler.transform(x_train))
    corrects = np.sum(train_preds == y_train_ohe_new.argmax(axis=1))
    train_acc = (corrects / x_train_new.shape[0]) * 100
    train_f1 = f1_score(y_train_ohe_new.argmax(axis=1), train_preds, average='weighted')
    accs['train'][name].append(train_acc)
    f1s['train'][name].append(train_f1)
    train_probs = clf.predict_proba(x_train_new)
    if verbose:
        print('Train acc: {}'.format(train_acc))
        print('Train F1: {}'.format(train_f1))
        print('CM \n')
        print(confusion_matrix(y_train_ohe_new.argmax(axis=1), train_preds))

    #svm_ = SVC(**best_params)
    #print(clf.best_params_)
    x_test = scaler.transform(x_test)
    test_preds = clf.predict(x_test)
    corrects = np.sum(test_preds == y_test_ohe.argmax(axis=1))
    test_acc = (corrects / x_test.shape[0]) * 100
    test_f1 = f1_score(y_test_ohe.argmax(axis=1), test_preds, average='weighted')
    accs['test'][name].append(test_acc)
    f1s['test'][name].append(test_f1)
    test_probs = clf.predict_proba(x_test)
    
    if verbose:
        print('Test acc: {}'.format(test_acc))
        print('Test F1: {}'.format(test_f1))
        print('CM \n')
        print(confusion_matrix(y_test_ohe.argmax(axis=1), test_preds))
    
    probs = {
        'train': train_probs,
        'test': test_probs
    }
    preds = {
        'train': train_preds,
        'test': test_preds
    }
    return accs, f1s, probs, preds

def get_val_set(x, y, classes, percentage = 0.1):
    np.random.seed(42)  
    x_train = np.array([]).reshape(0,x.shape[1])
    y_train = np.array([]).reshape(0,y.shape[1])
    x_val = np.array([]).reshape(0,x.shape[1])
    y_val = np.array([]).reshape(0,y.shape[1])
    for c in classes:
        indexes = np.where(y.argmax(axis=1) == c)[0]
        np.random.shuffle(indexes)
        len_val = int(percentage * len(indexes))
        len_train = len(indexes) - len_val
        index_train = indexes[0:len_train]
        index_val = indexes[len_train:]
        x_train = np.concatenate([x_train, x[index_train,...]], axis=0)
        y_train = np.concatenate([y_train, y[index_train]], axis=0)
        x_val = np.concatenate([x_val, x[index_val,...]], axis=0)
        y_val = np.concatenate([y_val, y[index_val]], axis=0)
    
    index_train = list(range(x_train.shape[0]))
    index_val = list(range(x_val.shape[0]))
    np.random.shuffle(index_train)
    np.random.shuffle(index_val)
    
    return x_train[index_train,...],y_train[index_train], x_val[index_val,...], y_val[index_val]

In [ ]:
from openpyxl import load_workbook
import re
path = '../Copy-Number-Variation/Splits_10CV/'
path_rna_mrmr = 'mrmrDEGs/'

n_genes = 6
writer_test = pd.ExcelWriter('results_excels/RNA_'+str(n_genes)+'gen_test_XGBoost.xlsx', engine='openpyxl') 
writer_train = pd.ExcelWriter('results_excels/RNA_'+str(n_genes)+'gen_train_XGBoost.xlsx', engine='openpyxl') 

accs = {
    'train': {'RNA': []},
    'test': {'RNA': []}
}
f1s = {
    'train': {'RNA': []},
    'test': {'RNA': []}
}

name = 'RNA'
for split in range(10):
    print(10*'-')
    print('Split {}/{}'.format(split,10))
    print(10*'-')
    
    print('Data read...')
    data = pd.read_csv(path_rna_mrmr+'mrmrDEGs_LC_3classes_split'+str(split)+'.csv')
    
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 

    train_final = []
    for i in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(train_cids),list(data['Case_IDs'].values)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for j in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(val_cids),list(data['Case_IDs'].values)[j])
        if resu:
            if resu.group(0) != '':
                val_final.append(j)

    #train_final.insert(0, 1)
    #val_final.insert(0, 1)
    df_train = data.iloc[train_final,]
    df_val = data.iloc[val_final,]

    case_ids_val = df_val['Case_IDs']
    #val_df_all = data_all[case_ids_val]
    y_val = all_rna['labelsAll'].loc[all_rna['Case_IDs'].isin(case_ids_val)].values
    #y_val = np.where(y_val == 'Blood Derived Normal', 'healthy', y_val)
    y_val = np.where(y_val == 'Healthy', 'healthy', y_val)
    y_val = np.where(y_val == 'Adenocarcinoma', 'adeno', y_val)
    y_val = np.where(y_val == 'Squamous', 'squa', y_val)
    
    case_ids_train = df_train['Case_IDs']
    #train_df_all = data_all[case_ids_train]
    y_train = all_rna['labelsAll'].loc[all_rna['Case_IDs'].isin(case_ids_train)].values
    #y_train = np.where(y_train == 'Blood Derived Normal', 'healthy', y_train)
    y_train = np.where(y_train == 'Healthy', 'healthy', y_train)
    y_train = np.where(y_train == 'Adenocarcinoma', 'adeno', y_train)
    y_train = np.where(y_train == 'Squamous', 'squa', y_train)
    
    x_train = df_train.iloc[:,1:n_genes+1].values
    x_val = df_val.iloc[:,1:n_genes+1].values
    y_train_ohe = ohe.transform(y_train.reshape(-1,1))
    y_val_ohe = ohe.transform(y_val.reshape(-1,1))
    print('End data read...')
    
    print('Training...')
    accs, f1s, probs, preds = train(x_train, x_val, y_train_ohe, y_val_ohe, accs, f1s, name, early=False, verbose=False)
    
    print("Saving SVM predictions... \n")
    
    sheet_name = 'split_'+str(split)
    
    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_val
    data['Preds'] = preds['test']
    data['Prob LUAD'] = probs['test'][:, 0]
    data['Prob HLT'] = probs['test'][:, 1]
    data['Prob LUSC'] = probs['test'][:, 2]
    data['Real'] = y_val_ohe.argmax(axis=1)
    data.to_excel(writer_test, sheet_name = sheet_name)

    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_train
    data['Preds'] = preds['train']
    data['Prob LUAD'] = probs['train'][:, 0]
    data['Prob HLT'] = probs['train'][:, 1]
    data['Prob LUSC'] = probs['train'][:, 2]
    data['Real'] = y_train_ohe.argmax(axis=1)
    data.to_excel(writer_train, sheet_name=sheet_name)

writer_train.close()
writer_test.close()

In [ ]:
print('Mean Acc in train: {}+-{}'.format(np.mean(accs['train']['RNA']),np.std(accs['train']['RNA'])))
print('Mean F1 in train: {}+-{}'.format(np.mean(f1s['train']['RNA'])*100,np.std(f1s['train']['RNA'])*100))
print(10*'-')
print('Mean Acc in test: {}+-{}'.format(np.mean(accs['test']['RNA']),np.std(accs['test']['RNA'])))
print('Mean F1 in test: {}+-{}'.format(np.mean(f1s['test']['RNA'])*100,np.std(f1s['test']['RNA'])*100))